# Yelp API - Lab



## Introduction 

Now that we've seen how the Yelp API works, it's time to put those API and SQL skills to work in order to do some basic business analysis! Taking things a step further, you'll also independently explore how to perform pagination in order to retrieve a full results set from the Yelp API!

## Objectives

You will be able to:
* Create a DB on AWS to store information from Yelp about businesses
* Create HTTP requests to get data from Yelp API
* Parse HTTP responses and insert the information into your DB
* Perform pagination to retrieve troves of data!
* Write SQL queries to answer questions about your data 

## Problem Introduction

You've now worked with some API calls, but we have yet to see how to retrieve a more complete dataset in a programmatic manner and combine it with our other data skills. In this lab you will get data from the Yelp API, store that data in a SQL Database on AWS, and write queries to answer follow-up questions. 


### Outline:

1. Determine which pieces of information you need to pull from the Yelp API.

2. Create a DB schema with 2 tables. One for the businesses and one for the reviews.

3. Create Python functions to:
  - Perform a search of businesses using pagination
  - Parse the API response for specific data points
  - Insert the data into your AWS DB

4. Use the functions above in a loop that will paginate over the results to retrieve all of the results. 

*Something might cause your code to break while it is running. You don't want to constantly repull the same data when this happens, so you should insert the data into the database as you call and parse it, not after you have all of the data*

5. Create functions to:
  - Retrieve the reviews data of one business
  - Parse the reviews response for specific review data
  - Insert the review data into the DB

6. Using SQL, query all of the business IDs. Using the 3 Python functions you've created, run your business IDs through a loop to get the reviews for each business and insert them into your DB.

7. Write SQL queries to answer the following questions about your data.


Bonus Steps:  
- Place your helper functions in a package so that your final notebook only has the major steps listed.
- Rewrite your business search functions to be able take an argument for the type of business you are searching for.
- Add another group of businesses to your database.


 
## SQL Questions:

- What are the 5 businesses with the highest average ratings?
- What are the 5 businesses with the lowest average ratings?
- What is the average rating of restaurants that have a price label of one dollar sign? Two dollar signs? Three dollar signs? 
- How many businesses have a rating greater than or  4.5?
- How many businesses have a rating less than 3?
- Return the text of the reviews for the most reviewed restaurant. 
- Return the name of the business with the most recent review. 
- Find the highest rated business and return text of the most recent review. If multiple business have the same rating, select the restaurant with the most reviews. 
- Find the lowest rated business and return text of the most recent review.  If multiple business have the same rating, select the restaurant with the least reviews. 


## Part I - Set up the DB

Start by reading SQL questions above to get an understanding of the data you will need. Then, read the documentation of Yelp API to understand what data you will receive in the response.  


Now that you are familiar with the data, create your SQL queries to create the DB and the appropriate tables. 

In [16]:
## Connect to DB server onAWS
import mysql.connector
from mysql.connector import errorcode
import pandas as pd
import config2

In [17]:
cnx = mysql.connector.connect(
    host = config2.host,
    user = config2.user,
    password = config2.passwd)

In [18]:
cursor = cnx.cursor()

In [20]:
## Create new DB 
#cursor.execute('''CREATE DATABASE yelp''')

In [80]:
# Create a table for the Businesses
DB_NAME = 'movie_DB'
TABLES = {}
TABLES['businesses'] = (
    "CREATE TABLE businesses ("
    "  business_id VARCHAR(30) NOT NULL,"
    "  name VARCHAR(50) NOT NULL,"
    "  price VARCHAR(10) ,"
    "  ave_rating DECIMAL(2,1),"
    "  review_count int,"
    "  PRIMARY KEY (business_id)"
    ") ENGINE=InnoDB")

TABLES['reviews'] = ('''
    CREATE TABLE reviews (
    review_id int NOT NULL AUTO_INCREMENT,
    review_text VARCHAR(1000),
    rating int NOT NULL,
    business_id VARCHAR(30) REFERENCES businesses(business_id),
    PRIMARY KEY (review_id) )''')

In [81]:
cnx = mysql.connector.connect(
    host = config2.host,
    user = config2.user,
    passwd = config2.passwd,
    database = db_name
)
cursor = cnx.cursor()

In [82]:
cursor.execute(TABLES['reviews'])

In [61]:
for table_name in TABLES:
    table_description = TABLES[table_name]
    try:
        print("Creating table {}: ".format(table_name), end='')
        cursor.execute(table_description)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("already exists.")
        else:
            print(err.msg)
    else:
        print("OK")

cursor.close()
cnx.close()

Creating table businesses: already exists.
Creating table reviews: You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near '"
    business_id VARCHAR(30) REFERENCES businesses(business_id)
    PRIMARY KEY' at line 4


In [ ]:
# Create a table for the reviews

## Part 2: Create ETL pipeline for the business data from the API

In [83]:
import requests

In [130]:
# write a function to make a call to the DB
def yelp_call(url_params, api_key,offset=0):
    url = 'https://api.yelp.com/v3/businesses/search'
    headers = {'Authorization' : 'Bearer {}'.format(api_key)}
    url_params['offset'] = offset
    response = requests.get(url, headers=headers, params=url_params)
    data = response.json()['businesses']
    return data



In [131]:
api = '-K4WtibH13OgRCP25mkLKBn20NqWGLvXx-DlaG9oKTfoiJSzHAJOlpI7y-VSJz6jZl1tXsBy-BOknfvbVVR2BHkZOTPXJZB9KXt2ViXHZjf9WpZk-qwEnMT4MIl6XXYx'
term = 'Sushi'
location = 'Manhattan NY'
url_params = {  'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                'limit' : 50
             }

In [92]:
businesses = yelp_call(url_params,api)

In [105]:
businesses[4].keys()

dict_keys(['id', 'alias', 'name', 'image_url', 'is_closed', 'url', 'review_count', 'categories', 'rating', 'coordinates', 'transactions', 'price', 'location', 'phone', 'display_phone', 'distance'])

In [116]:
# write a function to parse the API response 
def parse(json):
    bus_list = []
    
    columns = ['id','name', 'price', 'rating', 'review_count']
    for i in range(len(businesses)):
        if all([column in businesses[i].keys() for column in columns]):
            dic = {}
            dic['id'] = businesses[i]['id']
            dic['name'] = businesses[i]['name']
            dic['price'] = businesses[i]['price']
            dic['rating'] = businesses[i]['rating']
            dic['review_count'] = businesses[i]['review_count']
            
            tple = (dic['id'],dic['name'],dic['price'],dic['rating'],dic['review_count'])
            bus_list.append(tple)
    return bus_list




    # so that you can easily insert the data in to the DB

In [117]:
bus = parse(businesses)

In [118]:
bus

[('HZWHBfeuP4Fb6y8b8uqnbA', 'Sushi Yasaka', '$$', 4.0, 1693),
 ('MmXZLVwWcYugi_TXSHmjeQ', 'Sushi by Bou Times Square', '$$$', 4.5, 472),
 ('6lUsFj0KCao678o9L7QwUw', 'Hatsuhana Sushi Restaurant', '$$$', 4.0, 1260),
 ('BPZfuPvSxCfoppU4lPFm0Q', 'DOMODOMO', '$$$', 4.5, 962),
 ('_IkMfWpOkQXRZtJAYJUJIw', 'Hama Japanese Cuisine', '$$', 4.0, 352),
 ('YoR6ouUwehLFnBQ7RAjPlw', 'Zawa Japanese', '$$$', 4.5, 89),
 ('p4W_z-zh96LHEAtQ8LRSVQ', 'Shinagawa', '$$', 4.5, 82),
 ('aK2ipD9K8-usBw_gkvvJuw', 'Nare Sushi', '$$', 4.0, 315),
 ('MGRo61aHFUeRlWhSbTv2NA', 'Sushi By M', '$$$', 4.5, 278),
 ('Zjdtcex73QIz2_MvEvDrFQ', 'MakiMaki', '$$', 4.0, 142),
 ('xFp3xSQ43J-i7nmJfNBsZA', 'Norikoh', '$$', 4.0, 512),
 ('BOz0qTUMtAfljd3Vn1rerQ', 'Pado', '$$', 4.5, 215),
 ('OLSSJIp8pu5UlugTF7tuHA', 'Ootoya Chelsea', '$$', 4.0, 1345),
 ('pNe8ZCkGTe4cXfKdOF7kYQ', 'Ise Restaurant', '$$', 4.5, 403),
 ('mESab-FqL5QqSVsnX72asQ', 'Zen Ramen & Sushi', '$$', 4.5, 1321),
 ('fePu8xy0ir_ikZcS_vVSBA', 'Asuka Sushi', '$$', 4.0, 898),


In [121]:
# Write a function to take your parsed data and insert it into the DB
def upload(upload_list):    
    statement = '''INSERT INTO businesses (business_id, name, price, ave_rating, review_count) VALUES (%s, %s, %s, %s, %s)'''
    cnx = mysql.connector.connect(
    host = config2.host,
    user = config2.user,
    passwd = config2.passwd,
    database = db_name)
    cursor = cnx.cursor()
    
    cursor.executemany(statement, upload_list)
    cnx.commit()

In [122]:
#upload(bus)

## Part 3: Create ETL pipeline for the restaurant review data from the API

In [138]:
# write a query to pull back all of the business ids 
cnx = mysql.connector.connect(
    host = config2.host,
    user = config2.user,
    passwd = config2.passwd,
    database = db_name)
cursor = cnx.cursor()
cursor.execute('''SELECT business_id FROM businesses''')
query = cursor.fetchall()

# you will need these ids to pull back the reviews for each restaurant

In [193]:
query

[('0ePn6TQSJQky4ueKeGQvpQ',),
 ('6lUsFj0KCao678o9L7QwUw',),
 ('6qK0MyhsXYLgMSPWZdleGA',),
 ('7EaiwMVaNdQibSs0yYkE5Q',),
 ('a7rM-AOM7_xi8AlWX-Thkg',),
 ('aK2ipD9K8-usBw_gkvvJuw',),
 ('AtUxlW7ltZb73ZXMwpkY_g',),
 ('BOz0qTUMtAfljd3Vn1rerQ',),
 ('BPZfuPvSxCfoppU4lPFm0Q',),
 ('CdHOwIO6OFNXZxPYEUts2g',),
 ('Clumryf360iOnwpZ2ML8aw',),
 ('CqQ2Og9kf-LhlUWtODalCQ',),
 ('diz6_4TVSP6EdWEZCZ0Acg',),
 ('et9pAFtSvNOct9Qcmtcf7g',),
 ('fePu8xy0ir_ikZcS_vVSBA',),
 ('FXFQJmhK6elesopXdhjvFQ',),
 ('Gb79W0yuQfJvu0V0SDC0Ag',),
 ('HZWHBfeuP4Fb6y8b8uqnbA',),
 ('klAhw3xLQi9GF1tf_HnS7w',),
 ('kwq1W0opBnOYKoUXWYyQ5g',),
 ('loXvaVYhzOO3xTYzntlVBg',),
 ('M9NNvpKlOWLxJbSPDTe0Lg',),
 ('mESab-FqL5QqSVsnX72asQ',),
 ('MGRo61aHFUeRlWhSbTv2NA',),
 ('MmXZLVwWcYugi_TXSHmjeQ',),
 ('MwtXOkRUndPU-ivbtLD-bQ',),
 ('nEkVibaTuIFEcTNtltFgJg',),
 ('NRJq2BNcbjqAyxUB_bw7gw',),
 ('OLSSJIp8pu5UlugTF7tuHA',),
 ('oyyubJoK0VZ2V3a5sVJULg',),
 ('p4W_z-zh96LHEAtQ8LRSVQ',),
 ('pNe8ZCkGTe4cXfKdOF7kYQ',),
 ('PTYUAAJbEmvIv4EsyRrcnA',),
 ('TBp9gdS

In [148]:
def yelp_call_review(url_params, api_key):
    my_table = []
    for i in query:
        url = f'https://api.yelp.com/v3/businesses/{i[0]}/reviews'
        headers = {'Authorization' : 'Bearer {}'.format(api_key)}
        response = requests.get(url, headers=headers, params=url_params)
        data = response.json()
        my_table.append(data)
    return my_table

In [162]:
table_rev = yelp_call_review(url_params, api)

In [183]:
# write a function that takes a business id 
# and makes a call to the API for reivews
# then parse out the relevant information
def parse_review(reviews):
    review_list = []
    for j in range(len(table_rev)):
        for i in range(len(table_rev[j]['reviews'])):
            dic = {}
            dic['text'] = table_rev[j]['reviews'][i]['text']
            dic['rating'] = table_rev[j]['reviews'][i]['rating']
            dic['id'] = table_rev[j]['reviews'][i]['id']

            tple = (dic['text'], dic['rating'], dic['id'])
            review_list.append(tple)
    return review_list

In [185]:
# write a function to insert the parsed data into the reviews table
parsed_review = parse_review(table_rev)

In [191]:
def upload_review(upload_list):    
    statement = '''INSERT INTO reviews (review_text, rating, business_id) VALUES (%s, %s, %s)'''
    cnx = mysql.connector.connect(
    host = config2.host,
    user = config2.user,
    passwd = config2.passwd,
    database = db_name)
    cursor = cnx.cursor()
    
    cursor.executemany(statement, upload_list)
    cnx.commit()

In [192]:
upload_review(parsed_review)

## Part 4: Write SQL queries that will answer the questions posed. 

###  Pagination

Returning to the Yelp API, the [documentation](https://www.yelp.com/developers/documentation/v3/business_search) also provides us details regarding the API limits. These often include details about the number of requests a user is allowed to make within a specified time limit and the maximum number of results to be returned. In this case, we are told that any request has a maximum of 50 results per request and defaults to 20. Furthermore, any search will be limited to a total of 1000 results. To retrieve all 1000 of these results, we would have to page through the results piece by piece, retriving 50 at a time. Processes such as these are often refered to as pagination.

Now that you have an initial response, you can examine the contents of the json container. For example, you might start with ```response.json().keys()```. Here, you'll see a key for `'total'`, which tells you the full number of matching results given your query parameters. Write a loop (or ideally a function) which then makes successive API calls using the offset parameter to retrieve all of the results (or 5000 for a particularly large result set) for the original query. As you do this, be mindful of how you store the data. 

**Note: be mindful of the API rate limits. You can only make 5000 requests per day, and APIs can make requests too fast. Start prototyping small before running a loop that could be faulty. You can also use time.sleep(n) to add delays. For more details see https://www.yelp.com/developers/documentation/v3/rate_limiting.**

***Below is sample code that you can use to help you deal with the pagination parameter.***

In [ ]:
# Your code here; use a function or loop to retrieve all the results from your original request
import time

def yelp_call(url_params, api_key):
    url = 'https://api.yelp.com/v3/businesses/search'
    headers = {'Authorization': 'Bearer {}'.format(api_key)}
    response = requests.get(url, headers=headers, params=url_params)
    
    data = response.json()['businesses']
    return data

def all_results(url_params, api_key):
    num = response.json()['total']
    print('{} total matches found.'.format(num))
    cur = 0
    results = []
    while cur < num and cur < 1000:
        url_params['offset'] = cur
        results.append(yelp_call(url_params, api_key))
        time.sleep(1) #Wait a second
        results += 50
    return df

term = 'pizza'
location = 'Astoria NY'
url_params = {  'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                'limit' : 50
             }
df = all_results(url_params, api_key)
print(len(df))
df.head()

### Sample SQL Query 

Below is a SQL query to create a table.  Additionally here is a link to create a table with a foreign key.

http://www.mysqltutorial.org/mysql-foreign-key/

```CREATE TABLE IF NOT EXISTS tasks (
    task_id INT AUTO_INCREMENT,
    title VARCHAR(255) NOT NULL,
    start_date DATE,
    due_date DATE,
    status TINYINT NOT NULL,
    priority TINYINT NOT NULL,
    description TEXT,
    PRIMARY KEY (task_id)
)  ENGINE=INNODB;```

SQL Questions:
What are the 5 businesses with the highest average ratings?

What are the 5 businesses with the lowest average ratings?

What is the average rating of restaurants that have a price label of one dollar sign? Two dollar signs? Three dollar signs?

How many businesses have a rating greater than or 4.5?

How many businesses have a rating less than 3?

Return the text of the reviews for the most reviewed restaurant.

Return the name of the business with the most recent review.

Find the highest rated business and return text of the most recent review. If multiple business have the same rating,
select the restaurant with the most reviews.

Find the lowest rated business and return text of the most recent review. If multiple business have the same rating, select the restaurant with the least reviews.

In [195]:
cnx = mysql.connector.connect(
    host = config2.host,
    user = config2.user,
    passwd = config2.passwd,
    database = db_name)
cursor = cnx.cursor()

In [205]:
cursor.execute('''SELECT MAX(ave_rating) FROM businesses''')
xpx= cursor.fetchall()

In [210]:
float(xpx[0][0])

4.5